In [1]:
#import all
import sys
sys.path.append("../lib")
from tqdm import tqdm

import numpy as np
import healpy as hp
import pymaster as nmt 
import pysm3
import time
from mpfit import mpfit
import mpfitlib as mpl
import scipy
#from Nearest_Positive_Definite import *
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.patheffects as path_effects
import scipy.stats as st
import basicfunc as func
import analys_lib as an
import simu_lib as sim
import pysm3.units as u

#sim params

r = 0
nside = 16
Npix = hp.nside2npix(nside)
N=20000 
lmax = nside*3-1
#lmax=850
scale = 10
Nlbin = 10
fsky = 0.7
dusttype = 0
syncrotype = 0
kw = ''
load=True


# instr param

ifreq=[0,9,21]
instr_name='litebird_full'
instr =  np.load("../lib/instr_dict/%s.npy"%instr_name,allow_pickle=True).item()
freq= instr['frequencies']
sens_P= instr['sens_P']
freq=freq[ifreq]
sens_P=sens_P[ifreq]
sigpix= sens_P/(np.sqrt((4*np.pi)/Npix*(60*180/np.pi)**2))
b = nmt.bins.NmtBin(nside=nside,lmax=lmax,nlb=Nlbin)
leff = b.get_effective_ells()
N_freqs =len(freq)
Ncross=int(N_freqs*(N_freqs+1)/2)

nucross = []
for i in range(0,N_freqs):
    for j in range(i,N_freqs):
        nucross.append(np.sqrt(freq[i]*freq[j]))
nucross = np.array(nucross)


#cmb
CLcmb_or=hp.read_cl('../CLsimus/Cls_Planck2018_r0.fits') #TT EE BB TE
DL_lens = leff*(leff+1)*b.bin_cell(CLcmb_or[2,2:lmax+3])/2/np.pi

#mask

mask = hp.read_map("../masks/mask_fsky%s_nside%s_aposcale%s.npy"%(fsky,nside,scale))

#call foreground sky
if dusttype==None and syncrotype==None:
    mapfg=np.zeros((N_freqs,2,Npix))
else:
    if dusttype==None:
        sky = pysm3.Sky(nside=512, preset_strings=['s%s'%syncrotype])#,'s%s'%synctype])
    if syncrotype==None:
    	sky = pysm3.Sky(nside=512, preset_strings=['d%s'%dusttype])#,'s%s'%synctype])
    if syncrotype!=None and dusttype!=None:
    	sky = pysm3.Sky(nside=512, preset_strings=['d%s'%dusttype,'s%s'%syncrotype])
    mapfg= np.array([sim.downgrade_map(sky.get_emission(freq[f] * u.GHz).to(u.uK_CMB, equivalencies=u.cmb_equivalencies(freq[f]*u.GHz)),nside_in=512,nside_out=nside) for f in range(len(freq))])
    mapfg=mapfg[:,1:]

#np.save("./test-sim-cov/map-test/mapfg.npy",mapfg)

# call cmb

--------------------------------------------------------------------------

  Local host:   login34
  Local device: mlx5_1
--------------------------------------------------------------------------


In [2]:
# create N sims
noisemaps= np.zeros((N,3,N_freqs,2,Npix))
mapcmb = np.zeros((N,N_freqs,2,Npix))

for k in tqdm(range(0,N)):
    for p in range(3):
        for i in range(N_freqs):
            noisemaps[k,p,i,0] =np.random.normal(0,sigpix[i],size=Npix)
            noisemaps[k,p,i,1] =np.random.normal(0,sigpix[i],size=Npix)
    
    mapcmb0= hp.synfast(CLcmb_or,nside,pixwin=False,new=True)
    mapcmb1 = np.array([mapcmb0 for i in range(N_freqs)])
    mapcmb[k] = mapcmb1[:,1:]


100%|██████████| 20000/20000 [00:44<00:00, 448.30it/s]


In [ ]:
# compute the spectra of N2 sims
N2=20000

#workspace
wsp_dc=[]
for i in range(0,N_freqs): 
    for j in range(i,N_freqs):
        w_dc = nmt.NmtWorkspace()
        if i != j :
            w_dc.compute_coupling_matrix(nmt.NmtField(mask, 1*mapfg[i],purify_e=False, purify_b=True), nmt.NmtField(mask,1*mapfg[j],purify_e=False, purify_b=True), b)
        if i==j :
            w_dc.compute_coupling_matrix(nmt.NmtField(mask, 1*mapfg[i],purify_e=False, purify_b=True), nmt.NmtField(mask, 1*mapfg[j],purify_e=False, purify_b=True), b)
        wsp_dc.append(w_dc)
 
wsp_dc=np.array(wsp_dc)

def computecross(mapauto1,mapauto2,mapcross1,mapcross2):
    CLcross=np.zeros((Ncross,len(leff)))
    z=0
    for i in range(0,N_freqs):
        for j in range(i,N_freqs):
            if i != j :
                CLcross[z]=np.array((sim.compute_master(nmt.NmtField(mask, 1*mapauto1[i],purify_e=False, purify_b=True), nmt.NmtField(mask, 1*mapauto2[j],purify_e=False, purify_b=True), wsp_dc[z]))[3])
            if i==j :
                CLcross[z]=np.array((sim.compute_master(nmt.NmtField(mask, 1*mapcross1[i],purify_e=False, purify_b=True), nmt.NmtField(mask, 1*mapcross2[j],purify_e=False, purify_b=True), wsp_dc[z]))[3])
            z = z +1
    return leff*(leff+1)*CLcross/2/np.pi

DLcross_coadd= np.zeros((N2,Ncross,len(leff)))
DLcross_cmbnoise= np.zeros((N2,Ncross,len(leff)))
DLcross_cmb= np.zeros((N2,Ncross,len(leff)))
DLcross_noise= np.zeros((N2,Ncross,len(leff)))
DLcross_noisexcmb= np.zeros((N2,Ncross,len(leff)))
DLcross_noisexfg= np.zeros((N2,Ncross,len(leff)))
DLcross_cmbxfg= np.zeros((N2,Ncross,len(leff)))

for k in tqdm(range(0,N2)):
    #addition du bruit aux cartes
    mapauto = mapfg  + noisemaps[k,0] + mapcmb[k]
    mapcross1 = mapfg  + noisemaps[k,1]*np.sqrt(2) + mapcmb[k]
    mapcross2 = mapfg  + noisemaps[k,2]*np.sqrt(2) + mapcmb[k]
    DLcross_coadd[k]= computecross(mapauto,mapauto,mapcross1,mapcross2)
    mapauto_cmbnoise =   noisemaps[k,0] + mapcmb[k]
    mapcross1_cmbnoise = noisemaps[k,1]*np.sqrt(2) + mapcmb[k]
    mapcross2_cmbnoise = noisemaps[k,2]*np.sqrt(2) + mapcmb[k]
    DLcross_cmbnoise[k]= computecross(mapauto_cmbnoise,mapauto_cmbnoise,mapcross1_cmbnoise,mapcross2_cmbnoise)
    DLcross_cmb[k]= computecross(mapcmb[k],mapcmb[k],mapcmb[k],mapcmb[k])
    DLcross_noise[k]= computecross(noisemaps[k,0]*np.sqrt(2),noisemaps[k,0]*np.sqrt(2),noisemaps[k,1]*np.sqrt(2),noisemaps[k,2]*np.sqrt(2))
    DLcross_noisexcmb[k]= computecross(noisemaps[k,0]*np.sqrt(2), mapcmb[k],noisemaps[k,1]*np.sqrt(2), mapcmb[k])
    DLcross_noisexfg[k]= computecross(noisemaps[k,0]*np.sqrt(2), mapfg,noisemaps[k,1]*np.sqrt(2), mapfg)
    DLcross_cmbxfg[k]= computecross(mapcmb[k], mapfg,mapcmb[k], mapfg)
DLcross_fg = computecross(mapfg,mapfg,mapfg,mapfg)

np.save("./DLcross_coadd.npy",DLcross_coadd)
np.save("./DLcross_cmbnoise.npy",DLcross_cmbnoise)
np.save("./DLcross_cmb.npy",DLcross_cmb)
np.save("./DLcross_noise.npy",DLcross_noise)
np.save("./DLcross_cmbnoise.npy",DLcross_cmbnoise)
np.save("./DLcross_noisexcmb.npy",DLcross_noisexcmb)
np.save("./DLcross_noisexfg.npy",DLcross_noisexfg)
np.save("./DLcross_cmbxfg.npy",DLcross_cmbxfg)
np.save("./DLcross_fg.npy",DLcross_fg)

 45%|████▌     | 9090/20000 [1:15:00<1:31:38,  1.98it/s]

In [ ]:
f=0
nucrosslabel = []
for i in range(0,N_freqs):
    for j in range(i,N_freqs):
        nucrosslabel.append('%sx%s'%(freq[i],freq[j]))
nucrosslabel = np.array(nucrosslabel)

font = {'size': 20}
matplotlib.rc('font', **font)
cmap   = plt.get_cmap('jet_r',402) #color map parameter
plt.figure(figsize=(15,12))

def ploterrbar(l,DL,legend):
    DL_mean=np.mean(DL, axis=0)
    DL_std=np.std(DL, axis=0)
    plt.errorbar(l,DL_mean[int(f)],yerr=DL_std[int(f)],fmt='.',markersize=20,label=legend)

ploterrbar(leff,DLcross_coadd,legend='coadd')
ploterrbar(leff-1,DLcross_cmbnoise,legend='cmb+noise')
ploterrbar(leff+2,DLcross_cmb,legend='cmb')
ploterrbar(leff+1,DLcross_noise,legend='noise')
ploterrbar(leff+2,DLcross_noisexcmb,legend='noisexcmb')
ploterrbar(leff+3,DLcross_noisexfg,legend='noisexfg')
ploterrbar(leff+4,DLcross_cmbxfg,legend='cmbxfg')
plt.loglog()
plt.plot(leff, DLcross_fg[f], color='black', linestyle = '--', lw=5, label='fg',zorder=90)
plt.plot(leff, DL_lens, color='black', linestyle = '--', lw=5, label='lensing',zorder=90)
plt.legend()
tick = 45/3
plt.ylabel(r"$\mathcal{D}_\ell(%s) \, \,  [\mu \,{\rm K}_{\rm CMB}^2]$"%nucrosslabel[f],labelpad=56)
plt.xlabel(r'$\ell$')
plt.show()

In [ ]:
N3=20000
def getcov(DL,ellbin,N):
    return np.cov(np.swapaxes(DLcross_cmb[:N,:,ellbin],0,1))

ellbin=2
cov_cmb=getcov(DLcross_cmb,ellbin,N3)
cov_add=getcov(DLcross_coadd,ellbin,N3)
cov_noise=getcov(DLcross_noise,ellbin,N3)
cov_noisexfg=getcov(DLcross_noisexfg,ellbin,N3)
cov_cmbxfg=getcov(DLcross_cmbxfg,ellbin,N3)
cov_noise=getcov(DLcross_noise,ellbin,N3)

def plotcov(cov,title=''):
    plt.figure(figsize=(10, 10))
    plt.imshow(np.log10(abs(cov)), cmap='viridis', vmin=-8, vmax=-2,aspect='equal')# Set color limits
    cbar = plt.colorbar( label='$\log_{10}(|\Sigma|)$', fraction=0.046, pad=0.04)  
    positions = np.arange(len(nucrosslabel))  # Positions des ticks
    plt.xticks(positions, nucrosslabel, rotation=45)  # Rotation pour éviter les chevauchements
    plt.yticks(positions, nucrosslabel)
    plt.title(title)
    plt.xlabel(r'$\nu_ix\nu_j$')
    plt.ylabel(r'$\nu_ix\nu_j$')
    plt.tight_layout()
    plt.show()
plotcov(cov_cmb,title='cmb')
plotcov(cov_coadd,title='fg+cmb+n')
plotcov(cov_noise,title='n')
plotcov(cov_noisexfg,title='fgxn')
plotcov(cov_cmbxfg,title='fgxcmb')

plt.figure(figsize=(15,10))
plt.plot(nucrosslabel,np.diag(cov_coadd),label='coadd')
plt.plot(nucrosslabel,np.diag(cov_cmb),label="cmb")
plt.plot(nucrosslabel,np.diag(cov_noise),label="noise")
plt.plot(nucrosslabel,np.diag(cov_noisexfg),label="noisexfg")
plt.plot(nucrosslabel,np.diag(cov_cmbxfg),label="cmbxfg")
plt.ylabel("Var(X)")
plt.legend()
plt.semilogy()
plt.show()

In [ ]:
plt.figure(figsize=(30, 30))
plt.imshow(getcov(DLcross_coadd,ellbin,500)/getcov(DLcross_coadd,ellbin,N3), cmap='viridis', aspect='auto', vmin=0, vmax=2)  # Set color limits
plt.colorbar(label='$\Sigma(N=500)/\Sigma(N=20K)$')
plt.xlabel(r'$\sqrt{\nu_i\nu_j}$')
plt.ylabel(r'$\sqrt{\nu_i\nu_j}$')
plt.tight_layout()